# Cálculo explícito de los polinomios de Hermite utilizando la fórmula de Rodrigues
**Juan Antonio Villegas Recio**
> Nota: Este Notebook debe ser ejecutado utilizando un 'kernel' de SageMath, preferiblemente con una versión igual o superior a la 9.3.

El objetivo es implimentar un método de cálculo explícito de los polinomios de Hermite utilizando la fórmula de Rodrigues. También se harán algunas mediciones de tiempos de ejecución para compararlas con los tiempos calculados con la RR3T.

Los polinomios de Hermite son ortogonales respecto a la función peso $\rho(x) = e^{-x^2}$ y verifican la ecuación de Pearson con $\sigma(x)=1$, $x\in\mathbb R$. Por tanto, la fórmula de Rodrigues de los polinomios de Hermite, tomando por convenio $B_n=(-1)^n$ es
$$
H_n(x)=(-1)^n e^{x^2}\frac{d^n}{dx^n}e^{x^{-2}}
$$
Creamos una función que recree esta fórmula, devolviendo un polinomio

In [39]:
def Rodrigues_H(n):
    Bn = (-1)**n
    rho_inv = e**(x**2)
    der = derivative(e**(-x**2),x,n)
    return (Bn*rho_inv*der).full_simplify()

Veámos los primeros 11 polinomios de Hermite, esta vez estandarizados como suelen ser presentados:

In [61]:
Npol = 11
for i in range(Npol):
    print(str(i) + ": " + str(Rodrigues_H(i)))

0: 1
1: 2*x
2: 4*x^2 - 2
3: 8*x^3 - 12*x
4: 16*x^4 - 48*x^2 + 12
5: 32*x^5 - 160*x^3 + 120*x
6: 64*x^6 - 480*x^4 + 720*x^2 - 120
7: 128*x^7 - 1344*x^5 + 3360*x^3 - 1680*x
8: 256*x^8 - 3584*x^6 + 13440*x^4 - 13440*x^2 + 1680
9: 512*x^9 - 9216*x^7 + 48384*x^5 - 80640*x^3 + 30240*x
10: 1024*x^10 - 23040*x^8 + 161280*x^6 - 403200*x^4 + 302400*x^2 - 30240


Recordemos los tiempos de cálculo que nos ofrecía el método que utilizaba la RR3T, los cuales eran elevados y crecían muy rápido con $n$. Comprobaremos si el cálculo mediante la fórmula de Rodrigues mejora o no a la RR3T. Para ello, y de manera análoga, haremos $10$ ejecuciones independientes del cálculo de cada polinomio y, mediante la media de los $10$ tiempos de ejecución encontraremos una estimación bastante acertada del tiempo medio de ejecución del cálculo de cada polinomio.

In [62]:
import time
Nejecuciones = 10
tiempos_calculo = [[] for _ in range(Npol)]
for i in range(Nejecuciones):
    for j in range(Npol):
        start_time = time.time()
        Rodrigues_H(j)
        tiempos_calculo[j].append(time.time() - start_time)

for i in range(Npol):
    print(str(i) + ": " + str(sum(tiempos_calculo[i])/Nejecuciones))

0: 0.008675885200500489
1: 0.010713315010070801
2: 0.012655282020568847
3: 0.01445021629333496
4: 0.017413878440856935
5: 0.021716094017028807
6: 0.024535393714904784
7: 0.019978260993957518
8: 0.02009847164154053
9: 0.0219860315322876
10: 0.023472046852111815


Como vemos, estos tiempos de cálculo son muchísimo menores que los calculados para la RR3T. De hecho, obsérvese el tiempo que tarda el programa en calcular el polinomio de Hermite de grado 500:

In [63]:
start_time = time.time()
Rodrigues_H(500)
time.time() - start_time

17.087850332260132

Teniendo en cuenta que la RR3T tardaba aproximadamente un minuto en calcular el polinomio de grado 15, la mejora es sustancial.

In [67]:
import time
Nejecuciones = 10
tiempos_calculo_Rodrigues = [[] for _ in range(Npol)]
tiempos_calculo_nativa = [[] for _ in range(Npol)]

for i in range(Nejecuciones):
    for j in range(Npol):
        start_time = time.time()
        Rodrigues_H(j)
        tiempos_calculo_Rodrigues[j].append(time.time() - start_time)
        
        start_time = time.time()
        hermite(j,x)(j)
        tiempos_calculo_nativa[j].append(time.time() - start_time)

for i in range(Npol):
    print(str(i) + "| " + str(sum(tiempos_calculo_Rodrigues[i])/Nejecuciones) + \
          " | " + str(sum(tiempos_calculo_nativa[i])/Nejecuciones))

0| 0.009253740310668945 | 0.00019750595092773437
1| 0.010477662086486816 | 0.00028862953186035154
2| 0.013608288764953614 | 0.0003504037857055664
3| 0.013358306884765626 | 0.0003786563873291016
4| 0.016212320327758788 | 0.0004178285598754883
5| 0.01828434467315674 | 0.0005173921585083008
6| 0.01862807273864746 | 0.000535273551940918
7| 0.019593262672424318 | 0.0005413055419921875
8| 0.030555224418640135 | 0.0006180524826049805
9| 0.0229586124420166 | 0.0006430625915527343
10| 0.024494528770446777 | 0.0006523370742797851


# Gráficas interactivas

In [6]:
from sage.repl.ipython_kernel.interact import interact

## Hermite

In [30]:
@interact
def Hermite_plot(n=(0,10), a=(-10,0,0.1), b=(0,10,0.1)):
    pol = Rodrigues_H(n)
    plot(pol, a, b, color='red').show()


Interactive function <function Hermite_plot at 0x7fc4b209b3a0> with 3 widgets
  n: IntSlider(value=5, descript…

## Laguerre

In [38]:
def Rodrigues_L(n, alpha):
    Bn = 1/factorial(n)
    rho_inv = pow(x,-alpha)*e**x
    der = derivative(x**(n+alpha)*e**(-x),x,n)
    return (Bn*rho_inv*der).full_simplify()

@interact
def Laguerre_plot(n=(0,10), alpha=(0,10), a=(0,1,0.1), b=(1,50,0.1)):
    pol = Rodrigues_L(n, alpha)
    plot(pol, x, a, b, color='red').show()

Interactive function <function Laguerre_plot at 0x7fc4b289aa60> with 4 widgets
  n: IntSlider(value=5, descrip…

## Jacobi

In [50]:
def Rodrigues_J(n, alpha, beta):
    Bn = (-1)**n/(2**n*factorial(n))
    rho_inv = pow(1-x, -alpha)*pow(1+x, -beta)
    der = derivative(pow(1-x, n+alpha)*pow(1+x, n+beta),x,n)
    return Bn*rho_inv*der

@interact
def plot_Jacobi(n=(0,10), alpha=(0,10), beta=(0,10), a=(-1,0,0.1), b=(0,1,0.1)):
    pol = Rodrigues_J(n, alpha, beta)
    plot(pol, x, a, b, color='red').show()

Interactive function <function plot_Jacobi at 0x7fc4b27c8160> with 5 widgets
  n: IntSlider(value=5, descripti…